##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Introdução ao Keras Tuner

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/keras_tuner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/keras/keras_tuner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

O Keras Tuner é uma biblioteca que ajuda a escolher o conjunto ideal de hiperparâmetros do seu programa TensorFlow. O processo de seleção do conjunto certo de hiperparâmetros para sua aplicação de aprendizado de máquina é chamado de *tunagem de hiperparâmetros* ou *hipertunagem*.

Os hiperparâmetros são as variáveis que governam o processo de treinamento e a topologia de modelos de aprendizado de máquina. Essas variáveis permanecem constantes durante o processo de treinamento e impactam diretamente o programa de aprendizado de máquina. Há dois tipos de hiperparâmetros:

1. **Hiperparâmetros do modelo**, que influenciam a seleção do modelo, como o número e a largura das camadas ocultas.
2. **Hiperparâmetros do algoritmo**, que influenciam a velocidade e qualidade do algoritmo de aprendizado, como a taxa de aprendizado para o Método do Gradiente Descendente Estocástico (SGD, na sigla em inglês) e o número de vizinhos mais próximos para um classificador K Vizinhos Mais Próximos (KNN, na sigla em inglês).

Neste tutorial, você usará o Keras Tuner para hipertunar uma aplicação de classificação de imagens.

## Configuração

In [ ]:
import tensorflow as tf
from tensorflow import keras

Instale e importe o Keras Tuner.

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

## Baixar e preparar o dataset

Neste tutorial, você usará o Keras Tuner para descobrir os melhores hiperparâmetros para um modelo de aprendizado de máquina que classifica imagens de roupas do [dataset Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist).

Carregue os dados.

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Definir o modelo

Quando você cria um modelo para fazer hipertunagem, também define o espaço de busca de hiperparâmetros, além da arquitetura do modelo. O modelo configurado para fazer hipertunagem é chamado de *hipermodelo*.

Há duas estratégias para definir um hipermodelo:

- Usando uma função de construção de modelos.
- Fazendo uma subclasse da classe `HyperModel` da API do Keras Tuner.

Além disso, você pude usar duas classes [HyperModel](https://keras.io/api/keras_tuner/hypermodels/) pré-definidas: [HyperXception](https://keras.io/api/keras_tuner/hypermodels/hyper_xception/) e [HyperResNet](https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/) para aplicações de visão computacional.

Neste tutorial, você usará uma função de construção de modelos para definir o modelo de classificação de imagens. Essa função retorna um modelo compilado e usa os hiperparâmetros que você define internamente para hipertunar do modelo.

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instanciar o tuner e fazer a hipertunagem

Instancie o tuner para fazer a hipertunagem. O Keras Tuner disponibiliza quatro tuners: `RandomSearch`, `Hyperband`, `BayesianOptimization` e `Sklearn`. Neste tutorial, você usará o tuner [Hyperband](https://arxiv.org/pdf/1603.06560.pdf).

Para instanciar o tuner Hyperband, você precisa especificar o hupermodelo, o `objective` (objetivo) de otimização e o número máximo de épocas para o treinamento (`max_epochs`).

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

O algoritmo de tunagem Hyperband usa alocação adaptativa de recursos e parada antecipada para convergir rapidamente para um modelo de alto desempenho. Isso é feito usando-se chaveamento similar ao de campeonatos esportivos. O algoritmo treina uma grande quantidade de modelos com algumas épocas e prossegue somente com os 50% modelos com melhor desempenho na próxima rodada. O Hyperband determina o número de modelos a serem treinados em uma chave computando 1 + log<sub><code>factor</code></sub>(`max_epochs`) e arredondando para o número inteiro mais próximo.

Crie um callback para interromper o treinamento antecipadamente após a perda de validação atingir um determinado valor.

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Execute a busca de hiperparâmetros. Os argumentos do método de busca são os mesmos usados para `tf.keras.model.fit`, além do callback acima.

In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

## Treinar o modelo

Encontre o número ideal de épocas para treinar o modelo com os hiperparâmetros obtidos pela busca.

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Instancie o hipermodelo novamente e treine-o usando o número ideal de épocas acima.

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Para concluir este tutorial, avalie o hipermodelo com os dados de teste.

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

O diretório `my_dir/intro_to_kt` contém checkpoints e logs detalhados para cada tentativa (configuração do modelo) executada durante a busca de hiperparâmetros. Se você executar novamente a busca de hiperparâmetros, o Keras Tuner utilizará o estado atual usando esses logs para retomar a busca. Para desativar esse comportamento, passe o argumento adicional `overwrite=True` ao instanciar o tuner.

## Resumo

Neste tutorial, você aprendeu a usar o Keras Tuner para tunar os hiperparâmetros de um modelo. Para saber mais sobre o Keras Tuner, confira os seguintes recursos:

- [Keras Tuner no blog do TensorFlow](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
- [Site do Keras Tuner](https://keras-team.github.io/keras-tuner/)

Verifique também o [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) (Painel HParams) no TensorBoard para tunar interativamente os hiperparâmetros do seu modelo.